In [486]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow

In [487]:
df= pd.read_csv('cust_satisfaction.csv')
df.head()

,Gender,Customer Type,Type of Travel,Class,satisfaction,Age,Flight Distance,Inflight entertainment,Baggage handling,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes
0,Male,Loyal Customer,Personal Travel,Eco Plus,neutral or dissatisfied,13,460,5,4,5,25,18.0
1,Male,disloyal Customer,Business travel,Business,neutral or dissatisfied,25,235,1,3,1,1,6.0
2,Female,Loyal Customer,Business travel,Business,satisfied,26,1142,5,4,5,0,0.0
3,Female,Loyal Customer,Business travel,Business,neutral or dissatisfied,25,562,2,3,2,11,9.0
4,Male,Loyal Customer,Business travel,Business,satisfied,61,214,3,4,3,0,0.0


In [488]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103904 entries, 0 to 103903
Data columns (total 12 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Gender                      103904 non-null  object 
 1   Customer Type               103904 non-null  object 
 2   Type of Travel              103904 non-null  object 
 3   Class                       103904 non-null  object 
 4   satisfaction                103904 non-null  object 
 5   Age                         103904 non-null  int64  
 6   Flight Distance             103904 non-null  int64  
 7   Inflight entertainment      103904 non-null  int64  
 8   Baggage handling            103904 non-null  int64  
 9   Cleanliness                 103904 non-null  int64  
 10  Departure Delay in Minutes  103904 non-null  int64  
 11  Arrival Delay in Minutes    103594 non-null  float64
dtypes: float64(1), int64(6), object(5)
memory usage: 9.5+ MB


In [489]:
df.isnull().sum()
df.dropna(inplace=True)


In [490]:

df.duplicated().sum()
df.drop_duplicates(inplace=True)

In [491]:
df["Customer Type"].value_counts()

Customer Type
Loyal Customer       84517
disloyal Customer    18905
Name: count, dtype: int64

In [492]:
loyal_customer = df[df["Customer Type"] == "Loyal Customer"]
disloyal_customer = df[df["Customer Type"] == "disloyal Customer"]

In [493]:
loyal_customer=loyal_customer.sample(20000)
loyal_customer.shape

(20000, 12)

In [494]:
balanced_df = pd.concat([loyal_customer, disloyal_customer], axis=0)
balanced_df.shape

(38905, 12)

In [495]:
cat_col= balanced_df.select_dtypes(include=['object'])
cat_col.head()

,Gender,Customer Type,Type of Travel,Class,satisfaction
92321,Female,Loyal Customer,Personal Travel,Eco,neutral or dissatisfied
45523,Male,Loyal Customer,Business travel,Eco Plus,neutral or dissatisfied
43627,Male,Loyal Customer,Personal Travel,Eco,neutral or dissatisfied
75494,Male,Loyal Customer,Personal Travel,Eco,neutral or dissatisfied
31772,Male,Loyal Customer,Business travel,Business,satisfied


In [496]:
num_col= balanced_df.select_dtypes(exclude=['object'])
num_col.head()

,Age,Flight Distance,Inflight entertainment,Baggage handling,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes
92321,57,1855,5,5,2,51,61.0
45523,60,370,3,2,3,0,0.0
43627,67,550,4,1,4,0,0.0
75494,19,1085,4,5,4,1,12.0
31772,42,413,4,4,3,0,0.0


In [497]:
pd.get_dummies(cat_col, drop_first=True).astype(int).head()

,Gender_Male,Customer Type_disloyal Customer,Type of Travel_Personal Travel,Class_Eco,Class_Eco Plus,satisfaction_satisfied
92321,0,0,1,1,0,0
45523,1,0,0,0,1,0
43627,1,0,1,1,0,0
75494,1,0,1,1,0,0
31772,1,0,0,0,0,1


In [498]:
# ## one hot encoding
# cat_col = pd.get_dummies(cat_col, drop_first=True).astype(int)           # drop_first=True avoids dummy variable trap
# cat_col

In [499]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
ohe= OneHotEncoder(drop="if_binary")
cat_col_encoded = ohe.fit_transform(cat_col).toarray()
cat_col_encoded

array([[0., 0., 1., ..., 1., 0., 0.],
       [1., 0., 0., ..., 0., 1., 0.],
       [1., 0., 1., ..., 1., 0., 0.],
       ...,
       [0., 1., 0., ..., 1., 0., 0.],
       [1., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 1., 0., 0.]])

In [500]:
column_name=list(ohe.get_feature_names_out())
column_name

['Gender_Male',
 'Customer Type_disloyal Customer',
 'Type of Travel_Personal Travel',
 'Class_Business',
 'Class_Eco',
 'Class_Eco Plus',
 'satisfaction_satisfied']

In [501]:
one_hot = pd.DataFrame(cat_col_encoded,columns=column_name)
one_hot.head()

,Gender_Male,Customer Type_disloyal Customer,Type of Travel_Personal Travel,Class_Business,Class_Eco,Class_Eco Plus,satisfaction_satisfied
0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1.0,0.0,1.0,0.0,1.0,0.0,0.0
3,1.0,0.0,1.0,0.0,1.0,0.0,0.0
4,1.0,0.0,0.0,1.0,0.0,0.0,1.0


In [502]:
one_hot= one_hot.reset_index(drop=True)
num_col= num_col.reset_index(drop=True)
# final_df=pd.concat([one_hot,num_col],axis=1)
# final_df.head()

In [503]:
final_df=pd.concat([one_hot,num_col],axis=1)
final_df.head()

,Gender_Male,Customer Type_disloyal Customer,Type of Travel_Personal Travel,Class_Business,Class_Eco,Class_Eco Plus,satisfaction_satisfied,Age,Flight Distance,Inflight entertainment,Baggage handling,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes
0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,57,1855,5,5,2,51,61.0
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,60,370,3,2,3,0,0.0
2,1.0,0.0,1.0,0.0,1.0,0.0,0.0,67,550,4,1,4,0,0.0
3,1.0,0.0,1.0,0.0,1.0,0.0,0.0,19,1085,4,5,4,1,12.0
4,1.0,0.0,0.0,1.0,0.0,0.0,1.0,42,413,4,4,3,0,0.0


In [504]:
## x and y --> train test split ---> algo traning

In [505]:
# matrix = final_df.corr()
# matrix       

In [506]:
## trainning and testing data
from sklearn.model_selection import train_test_split
## divide the data into x and y or independent and dependent variable
x =final_df.drop("Customer Type_disloyal Customer",axis=1)
y= final_df["Customer Type_disloyal Customer"]
x_train,x_test,y_train,y_test=train_test_split(x,y,
                                                test_size=0.2) 

In [507]:
# Deep Learning

from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train_scaled=sc.fit_transform(x_train)
x_test_scaled=sc.transform(x_test)

In [508]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense      # dense is work of InputLayer outputLayer and hidden layer

In [509]:
x_train.shape[1]

13

In [510]:
## define your ANN model
model = Sequential()
# input layer
## 68 --> no of neurons --> genral connverdation -- 128
model.add(Dense(68, activation='relu', input_dim=(x_train_scaled.shape[1])))  # input layer with 68 neurons and relu activation function  
# hidden layer
model.add(Dense(32, activation='relu'))  # hidden layer with 32 neurons and relu activation function
model.add(Dense(24, activation='relu'))  # hidden layer with 32 neurons and relu activation function
model.add(Dense(12, activation='relu'))  # hidden layer with 32 neurons and relu activation function 
# output layeer
model.add(Dense(1, activation='sigmoid'))  # output layer with 1 neuron and sigmoid activation function for binary classification

## compile the model
model.compile(optimizer='adam',
               loss='binary_crossentropy',
                 metrics=['accuracy'])  # in compie we define the optimizer, loss function and metrics

model.summary()  # to see the summary of the model

C:\Users\Akshat\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_60 (Dense)                │ (None, 68)             │           952 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_61 (Dense)                │ (None, 32)             │         2,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_62 (Dense)                │ (None, 24)             │           792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_63 (Dense)                │ (None, 12)             │           300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_64 (Dense)                │ (None, 1)              │            13 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,265 (16.66 KB)

 Trainable params: 4,265 (16.66 KB)

 Non-trainable params: 0 (0.00 B)

In [511]:
(13+1)*68 

952

In [512]:
history= model.fit(x_train_scaled, y_train, 
                   epochs=10,
                     validation_data=(x_test_scaled,y_test))  # training the model with 100 epochs and batch size of 32 and validation split of 0.2

Epoch 1/10
973/973 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.8547 - loss: 0.3479 - val_accuracy: 0.9104 - val_loss: 0.2391
Epoch 2/10
973/973 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9075 - loss: 0.2395 - val_accuracy: 0.9135 - val_loss: 0.2264
Epoch 3/10
973/973 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9106 - loss: 0.2270 - val_accuracy: 0.9165 - val_loss: 0.2174
Epoch 4/10
973/973 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9158 - loss: 0.2183 - val_accuracy: 0.9195 - val_loss: 0.2154
Epoch 5/10
973/973 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9176 - loss: 0.2123 - val_accuracy: 0.9198 - val_loss: 0.2118
Epoch 6/10
973/973 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9172 - loss: 0.2141 - val_accuracy: 0.9221 - val_loss: 0.2057
Epoch 7/10
973/973 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9209 - loss: 0.2025 - val_accuracy: 0.9229 - val_loss: 0.2040
Epoch 8/10
973/973 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9235 - loss: 0.2005 - val_accuracy: 0.

In [513]:
## prediction
y_pred = model.predict(x_test_scaled)  # predicting the test data
prediction_label=(y_pred>0.5).astype(int).ravel()  # converting the predicted values to 0 or 1 based on the threshold of 0.5
prediction_label


244/244 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


array([1, 0, 0, ..., 1, 0, 1])

In [514]:
model.save('model.h5')  # saving the model



In [515]:
from tensorflow.keras.models import load_model
model_load= load_model('model.h5')  # loading the saved model